In [1]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from IPython.display import display, IFrame, HTML

# Controller Equations

\begin{gather}
\dot{x} = Ax + Bu \\
A = \frac{\partial f}{\partial x} = \frac{\partial f}{\partial s} \quad\quad B = \frac{\partial f}{\partial u} = \frac{\partial f}{\partial i} \\
u = -Kx \\
x = s - s_{eq} \quad\quad u = i - i_{eq} \\
s = [o_x, o_y, o_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z] \\
u = [\tau_x, \tau_y, \tau_z, f_z]
\end{gather}

In [2]:
o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z = sym.S("o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z")
tau_x, tau_y, tau_z, f_z = sym.S("tau_x, tau_y, tau_z, f_z")

\begin{gather}
s_{eq} = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] \\
i_{eq} = [0, 0, 0, g]
\end{gather}

In [3]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]

In [4]:
x = []
u = []

# 2. Derive models

## 2.1 Define symbolic variables

Define states.

In [5]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

Define inputs.

In [6]:
# gyroscope measurements - components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# z-axis accelerometer measurement - specific force (meters / second^2)
a_z = sym.symbols('a_z')

Define outputs.

In [7]:
d0, d1, d2, d3, d4, d5, d6, d7, n_x, n_y, r = sym.symbols('d0, d1, d2, d3, d4, d5, d6, d7,n_x, n_y, r')
a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 = sym.symbols('a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 ')

Define parameters.

In [8]:
g, k_flow = sym.symbols('g, k_flow')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [9]:
v_01in1 = sym.Matrix([[v_x], [v_y], [v_z]])
w_01in1 = sym.Matrix([[w_x], [w_y], [w_z]])

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [10]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [11]:
R_1in0 = Rz * Ry * Rx

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [12]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [13]:
N = sym.simplify(Ninv.inv())

## 2.3 Derive equations of motion

Ratio of net thrust to mass in terms of z-axis accelerometer measurement.

In [14]:
f_z_over_m = a_z + (w_01in1.cross(v_01in1))[2]

Ratio of forces to mass.

In [15]:
f_in1_over_m = R_1in0.T * sym.Matrix([[0], [0], [-g]]) + sym.Matrix([[0], [0], [f_z_over_m]])

Equations of motion.

In [16]:
f = sym.Matrix.vstack(
    R_1in0 * v_01in1,
    N * w_01in1,
    (f_in1_over_m - w_01in1.cross(v_01in1)),
)

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

where

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [17]:
f

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                          g*sin(theta) + v_y*w_z - v_z*w_y

# Observer Equations

In [18]:
o_x, o_y, o_z = sym.S("o_x, o_y, o_z") # drone position in abosulte frame -> loco pos frame
x_0, y_0, z_0 = sym.S("x_0, y_0, z_0")
x_1, y_1, z_1 = sym.S("x_1, y_1, z_1")
x_2, y_2, z_2 = sym.S("x_2, y_2, z_2")
x_3, y_3, z_3 = sym.S("x_3, y_3, z_3")
x_4, y_4, z_4 = sym.S("x_4, y_4, z_4")
x_5, y_5, z_5 = sym.S("x_5, y_5, z_5")
x_6, y_6, z_6 = sym.S("x_6, y_6, z_6")
x_7, y_7, z_7 = sym.S("x_7, y_7, z_7")

In [19]:
psi, theta, phi = sym.S("psi, theta, phi")
v_x, v_y, v_z = sym.S("v_x, v_y, v_z")

In [20]:
w_x, w_y, w_z = sym.S("w_x, w_y, w_z")
a_z = sym.S("a_z")

In [21]:
n_x, n_y, r = sym.S("n_x, n_y, r")

In [22]:
g, k_flow = sym.S("g, k_flow")

In [23]:
anchor_pos_0 = sym.sqrt((o_x-x_0)**2 + (o_y-y_0)**2 + (o_z-z_0)**2)
anchor_pos_1 = sym.sqrt((o_x-x_1)**2 + (o_y-y_1)**2 + (o_z-z_1)**2)
anchor_pos_2 = sym.sqrt((o_x-x_2)**2 + (o_y-y_2)**2 + (o_z-z_2)**2)
anchor_pos_3 = sym.sqrt((o_x-x_3)**2 + (o_y-y_3)**2 + (o_z-z_3)**2)
anchor_pos_4 = sym.sqrt((o_x-x_4)**2 + (o_y-y_4)**2 + (o_z-z_4)**2)
anchor_pos_5 = sym.sqrt((o_x-x_5)**2 + (o_y-y_5)**2 + (o_z-z_5)**2)
anchor_pos_6 = sym.sqrt((o_x-x_6)**2 + (o_y-y_6)**2 + (o_z-z_6)**2)
anchor_pos_7 = sym.sqrt((o_x-x_7)**2 + (o_y-y_7)**2 + (o_z-z_7)**2)

In [24]:
h = sym.Matrix([
    anchor_pos_0,
    anchor_pos_1,
    anchor_pos_2,
    anchor_pos_3,
    anchor_pos_4,
    anchor_pos_5,
    anchor_pos_6,
    anchor_pos_7,
    k_flow*(v_x-o_z*w_y)/o_z, # n_x
    k_flow*(v_y+o_z*w_x)/o_z, # n_y
    o_z / (sym.cos(phi)*sym.cos(theta)) # r
])
h

Matrix([
[sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2)],
[sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2)],
[sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2)],
[sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2)],
[sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2)],
[sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2)],
[sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2)],
[sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2)],
[                           k_flow*(-o_z*w_y + v_x)/o_z],
[                            k_flow*(o_z*w_x + v_y)/o_z],
[                             o_z/(cos(phi)*cos(theta))]])

In [25]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]
o = [d0,
    d1,
    d2,
    d3,
    d4,
    d5,
    d6,
    d7,
    n_x,
    n_y,
    r,
    ]
p = [g, k_flow]


In [26]:
o_x_eq = sym.symbols('o_x_eq')
o_y_eq = sym.symbols('o_y_eq')
o_z_eq = sym.symbols('o_z_eq')
s_eq = [o_x_eq, o_y_eq, o_z_eq, 0, 0, 0, 0, 0, 0]
i_eq = [0, 0, 0, g]

s_eq = [sym.nsimplify(a) for a in s_eq]
i_eq = [sym.nsimplify(a) for a in i_eq]


In [27]:
x = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]) - sym.Matrix(s_eq)
u = sym.Matrix([w_x, w_y, w_z, a_z]) - sym.Matrix(i_eq)
y = sym.Matrix(o) - h.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
y

Matrix([
[d0 - sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[d1 - sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[d2 - sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2)],
[d3 - sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2)],
[d4 - sqrt((o_x_eq - x_4)**2 + (o_y_eq - y_4)**2 + (o_z_eq - z_4)**2)],
[d5 - sqrt((o_x_eq - x_5)**2 + (o_y_eq - y_5)**2 + (o_z_eq - z_5)**2)],
[d6 - sqrt((o_x_eq - x_6)**2 + (o_y_eq - y_6)**2 + (o_z_eq - z_6)**2)],
[d7 - sqrt((o_x_eq - x_7)**2 + (o_y_eq - y_7)**2 + (o_z_eq - z_7)**2)],
[                                                                 n_x],
[                                                                 n_y],
[                                                         -o_z_eq + r]])

In [28]:
A = f.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
B = f.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
A, B

(Matrix([
 [0, 0, 0, 0, 0,  0, 1, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 1, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 1],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, g,  0, 0, 0, 0],
 [0, 0, 0, 0, 0, -g, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0]]),
 Matrix([
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 1]]))

In [29]:

C = h.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
D = h.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))



In [30]:
s_obs_index = [0, 1, 2, 4, 5, 6, 7, 8]
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D
C_obs

Matrix([
[(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2), (o_y_eq - y_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq

In [31]:
x_obs = sym.Matrix([o_x, o_y, o_z - o_z_eq, theta, phi, v_x, v_y, v_z])
C_obs*x_obs + D_obs*u - y

Matrix([
[-d0 + o_x*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + o_y*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[-d1 + o_x*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + o_y*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[-d2 + o_x*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + o_y*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)*

In [32]:
anchors_x_sym = [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7]
anchors_y_sym = [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7]
anchors_z_sym = [z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7]
anchors_x = [-1.82, -3.57, 1.82, 2.67, -3.61, 1.84, 2.74, -1.85]
anchors_y = [2.18, 3.18, 2.21, -3.30, -3.30, -0.90, 3.18, -0.93]
anchors_z = [1.10, 2.24, 1.10, 2.20, 2.25, 1.10, 2.24, 1.09]

In [33]:
C_obs = C_obs.subs(tuple(zip(anchors_x_sym, anchors_x))).subs(tuple(zip(anchors_y_sym, anchors_y))).subs(tuple(zip(anchors_z_sym, anchors_z)))
C_obs = C_obs.subs(o_x_eq, 0).subs(o_y_eq, 0).subs(o_z_eq, 0.5).subs(k_flow, 0.01 * 30.0 / np.deg2rad(4.2))
C_obs


Matrix([
[ 0.627034718512852, -0.751063563932977, -0.206714742366874, 0, 0,                0,                0, 0],
[ 0.701689487884987, -0.625034333746291, -0.341999918464952, 0, 0,                0,                0, 0],
[-0.622190406739018, -0.755516922468808, -0.205117716507369, 0, 0,                0,                0, 0],
[-0.583909671379651,   0.72168611069395, -0.371777693387793, 0, 0,                0,                0, 0],
[ 0.694943039652432,  0.635266490541004, -0.336883744983866, 0, 0,                0,                0, 0],
[-0.862074960391653,  0.421667100191569, -0.281111400127713, 0, 0,                0,                0, 0],
[-0.602997864002159, -0.699829637783527, -0.382925650862685, 0, 0,                0,                0, 0],
[ 0.859257706054903,  0.431951171151924,  -0.27403353868778, 0, 0,                0,                0, 0],
[                 0,                  0,                  0, 0, 0, 8.18511135901176,                0, 0],
[                 0,        

In [34]:
A_arr = np.array(A_obs.subs(g, -9.81).tolist(), dtype=np.float64)
B_arr = np.array(B_obs.subs(g, -9.81).tolist(), dtype=np.float64)
C_arr = np.array(C_obs.tolist(), dtype=np.float64)

In [35]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K
Q = np.diag([
    1/0.049**2,
    1/0.072**2,
    1/0.046**2,
    1/0.114**2,
    1/0.063**2,
    1/0.048**2,
    1,
    1/0.038**2,
    1/1.696**2,             # n_x
    1/1.738**2,             # n_y
    1/0.148**2,             # r
])

R = np.diag([
    1/0.094**2,
    1/0.111**2,
    1/0.009**2, # o_z
    1/0.036**2, # theta
    1/0.030**2, # phi
    1/0.176**2, # v_x
    1/0.178**2, # v_y
    1/0.354**2, # v_z
])
L_1 = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T
print(L_1.tolist())


[[1.0612354567973687, 0.5120515444032716, -1.227422341494915, -0.23602589430824622, 0.5720823035122847, -1.743479318219069, -0.0027509951457110617, 2.13496862753227, 0.022054950986334473, -0.0009918237959717557, 0.05568053447424029], [-1.8265623351877684, -0.7216832159932789, -1.914327947922793, 0.31356160012248663, 0.8156385587324054, 1.0919070460636557, -0.0037896847062179563, 1.440080240130129, -0.0011664347360733967, 0.023748065969326774, 0.010696428477186657], [-1.0308784680767624, -0.8410014443671401, -1.8790640480913656, -0.46580007967827997, -1.0070209845052163, -2.221890807147945, -0.006575106275751487, -2.016991927839532, 0.00390442604268722, 0.0006624141354791948, 0.6766781347187489], [-0.19119621603323952, -0.0960446261344245, 0.20943001664848512, 0.036544018295334525, -0.11451841840016151, 0.2860702931259912, 0.0004457391446861992, -0.40492827518313934, -0.014105118985698283, 0.0001684160760758503, -0.0037822735957417164], [-0.2285348991646402, -0.08941836278437573, -0.245

In [36]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K
Q = np.diag([
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,             # n_x
    1,             # n_y
    1,             # r
])

R = np.diag([
    1,
    1,
    1, # o_z
    1, # theta
    1, # phi
    1, # v_x
    1, # v_y
    1, # v_z
])
L_1 = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T
print(L_1.tolist())


[[0.3180489943470645, 0.35534433377172314, -0.31585157789868495, -0.29821898145092024, 0.3509203825098955, -0.43880375885054035, -0.30679594936681986, 0.43469067472125306, 0.11790331190534478, -9.654711870144457e-06, 0.003590744459261043], [-0.41404335035890094, -0.34233976127975285, -0.41552483450602606, 0.40579375856518957, 0.3563582152240883, 0.2384101217191572, -0.3821835054177546, 0.24262165863317836, -9.665354709109912e-06, 0.11849182375122658, -0.013955880783085102], [-0.2346526027361412, -0.3980459263629872, -0.23716485961905617, -0.45709340794356407, -0.4095359783329232, -0.34540030810949546, -0.45066476308547887, -0.3308925346291106, 3.6670916123952184e-05, -0.00012857175039797036, 1.1967500474500083], [-0.0013389045963223254, -0.0014997924714295298, 0.0013278869703701028, 0.0012415335824376597, -0.0014879763763421447, 0.0018368121665299935, 0.0012851484950960482, -0.0018377365930699315, -0.999991018132803, 3.604321715027406e-08, 9.285775006909661e-06], [-0.001515642166087859

In [37]:
L_str = np.array2string(L_1,
                        formatter={'float_kind': lambda x: f'{x:12.4f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'L = {L_str}')

L = [[      0.3180       0.3553      -0.3159      -0.2982       0.3509      -0.4388      -0.3068       0.4347       0.1179      -0.0000       0.0036]
     [     -0.4140      -0.3423      -0.4155       0.4058       0.3564       0.2384      -0.3822       0.2426      -0.0000       0.1185      -0.0140]
     [     -0.2347      -0.3980      -0.2372      -0.4571      -0.4095      -0.3454      -0.4507      -0.3309       0.0000      -0.0001       1.1968]
     [     -0.0013      -0.0015       0.0013       0.0012      -0.0015       0.0018       0.0013      -0.0018      -1.0000       0.0000       0.0000]
     [     -0.0015      -0.0013      -0.0015       0.0014       0.0013       0.0008      -0.0014       0.0009      -0.0000       1.0000       0.0000]
     [      0.0090       0.0101      -0.0090      -0.0084       0.0100      -0.0124      -0.0087       0.0124       1.8429      -0.0000       0.0000]
     [     -0.0109      -0.0090      -0.0109       0.0105       0.0092       0.0061      -0.0101    

In [38]:
A_sym = sym.Matrix(A)
B_sym = sym.Matrix(B)
A_obs_sym = sym.Matrix(A_obs)
B_obs_sym = sym.Matrix(B_obs)
C_obs_sym = sym.Matrix(C_obs)
D_obs_sym = sym.Matrix(D_obs)
# L_0, L_1, L_2, L_3, L_4, L_5 = sym.symbols('L_0, L_1, L_2, L_3, L_4, L_5')
# L_sym = sym.Matrix([[0, 0, L_0],
#                     [L_1, 0, 0],
#                     [0, L_2, 0],
#                     [L_3, 0, 0],
#                     [0, L_4, 0],
#                     [0, 0, L_5]])
L_sym = sym.Matrix(L_1)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')
psi, theta, phi = sym.symbols('psi, theta, phi')
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
a_z, g, n_x, n_y, r = sym.symbols('a_z, g, n_x, n_y, r')
x_sym = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z])
x_obs_sym = sym.Matrix([o_x, o_y, o_z, theta, phi, v_x, v_y, v_z])
u_sym = sym.Matrix([w_x, w_y, w_z, a_z - g])
y_sym = sym.Matrix([a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7, n_x, n_y, r])
x_dot_sym = A_sym * x_sym + B_sym * u_sym
x_obs_dot_sym = A_obs_sym * x_obs_sym + B_obs_sym * u_sym
y_err_sym = C_obs_sym * x_obs_sym + D_obs_sym * u_sym - y_sym
x_hat_dot_sym = x_obs_dot_sym - L_sym * y_err_sym

In [39]:
Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10 = sym.symbols('Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10')
Y_err = sym.Matrix([Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10])
output_list = (x_obs_dot_sym - L_sym * Y_err).tolist()
output_list

[[-0.318048994347065*Y_0 - 0.355344333771723*Y_1 - 0.00359074445926104*Y_10 + 0.315851577898685*Y_2 + 0.29821898145092*Y_3 - 0.350920382509895*Y_4 + 0.43880375885054*Y_5 + 0.30679594936682*Y_6 - 0.434690674721253*Y_7 - 0.117903311905345*Y_8 + 9.65471187014446e-6*Y_9 + v_x],
 [0.414043350358901*Y_0 + 0.342339761279753*Y_1 + 0.0139558807830851*Y_10 + 0.415524834506026*Y_2 - 0.40579375856519*Y_3 - 0.356358215224088*Y_4 - 0.238410121719157*Y_5 + 0.382183505417755*Y_6 - 0.242621658633178*Y_7 + 9.66535470910991e-6*Y_8 - 0.118491823751227*Y_9 + v_y],
 [0.234652602736141*Y_0 + 0.398045926362987*Y_1 - 1.19675004745001*Y_10 + 0.237164859619056*Y_2 + 0.457093407943564*Y_3 + 0.409535978332923*Y_4 + 0.345400308109495*Y_5 + 0.450664763085479*Y_6 + 0.330892534629111*Y_7 - 3.66709161239522e-5*Y_8 + 0.00012857175039797*Y_9 + v_z],
 [0.00133890459632233*Y_0 + 0.00149979247142953*Y_1 - 9.28577500690966e-6*Y_10 - 0.0013278869703701*Y_2 - 0.00124153358243766*Y_3 + 0.00148797637634214*Y_4 - 0.00183681216652

In [40]:
for i in range(len(output_list)):
    line = str(output_list[i])
    output_list[i] = line.replace("*", "f*" )
    print(output_list[i])
    print("\n")



[-0.318048994347065f*Y_0 - 0.355344333771723f*Y_1 - 0.00359074445926104f*Y_10 + 0.315851577898685f*Y_2 + 0.29821898145092f*Y_3 - 0.350920382509895f*Y_4 + 0.43880375885054f*Y_5 + 0.30679594936682f*Y_6 - 0.434690674721253f*Y_7 - 0.117903311905345f*Y_8 + 9.65471187014446e-6f*Y_9 + v_x]


[0.414043350358901f*Y_0 + 0.342339761279753f*Y_1 + 0.0139558807830851f*Y_10 + 0.415524834506026f*Y_2 - 0.40579375856519f*Y_3 - 0.356358215224088f*Y_4 - 0.238410121719157f*Y_5 + 0.382183505417755f*Y_6 - 0.242621658633178f*Y_7 + 9.66535470910991e-6f*Y_8 - 0.118491823751227f*Y_9 + v_y]


[0.234652602736141f*Y_0 + 0.398045926362987f*Y_1 - 1.19675004745001f*Y_10 + 0.237164859619056f*Y_2 + 0.457093407943564f*Y_3 + 0.409535978332923f*Y_4 + 0.345400308109495f*Y_5 + 0.450664763085479f*Y_6 + 0.330892534629111f*Y_7 - 3.66709161239522e-5f*Y_8 + 0.00012857175039797f*Y_9 + v_z]


[0.00133890459632233f*Y_0 + 0.00149979247142953f*Y_1 - 9.28577500690966e-6f*Y_10 - 0.0013278869703701f*Y_2 - 0.00124153358243766f*Y_3 + 0.001

In [41]:
x_dot_sym

Matrix([
[    v_x],
[    v_y],
[    v_z],
[    w_z],
[    w_y],
[    w_x],
[g*theta],
[ -g*phi],
[a_z - g]])

In [42]:
x_hat_dot_sym


Matrix([
[                               0.318048994347065*a_0 + 0.355344333771723*a_1 - 0.315851577898685*a_2 - 0.29821898145092*a_3 + 0.350920382509895*a_4 - 0.43880375885054*a_5 - 0.30679594936682*a_6 + 0.434690674721253*a_7 + 9.65471187014446e-6*k_flow*w_x + 0.117903311905345*k_flow*w_y + 0.117903311905345*n_x - 9.65471187014446e-6*n_y - 2.0000819388963*o_x - 0.00280219058557163*o_y + 0.00453090403648485*o_z + 0.00359074445926104*r + 0.0349482624584559*v_x + 7.90248917963051e-5*v_y],
[                              -0.414043350358901*a_0 - 0.342339761279753*a_1 - 0.415524834506026*a_2 + 0.40579375856519*a_3 + 0.356358215224088*a_4 + 0.238410121719157*a_5 - 0.382183505417755*a_6 + 0.242621658633178*a_7 - 0.118491823751227*k_flow*w_x - 9.66535470910991e-6*k_flow*w_y - 9.66535470910991e-6*n_x + 0.118491823751227*n_y - 0.00280453047535992*o_x - 1.83091493368707*o_y - 0.015869894477488*o_z - 0.0139558807830851*r + 7.91120046184133e-5*v_x + 0.0301312274638159*v_y],
[                   -0.